In [ ]:
import os
import re
import ast
import json
import time

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

from langchain_core.tools import tool

from langchain_openai import ChatOpenAI

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

from IPython.display import Image, display, clear_output


In [24]:
@tool
def get_storage_locations_in_plant(plant: str) -> str: # fixed function
    """
    Requests storage locations from the system.
    Args:
        plant (str): The plant to get storage locations for.
    Returns:
        str: A list of storage location ids and their descriptions.
    """
    
    x = global_input_function(
        {"tool": "get_storage_locations_in_plant", "plant": plant}
    )
    if x == "error":
        raise
    return x

@tool
def get_pollex_storage_locations() -> str: # fixed function
    """
    Requests pollex storage locations.
    Returns:
        str: A list of storage location ids and their descriptions.
    """
    
    x = global_input_function(
        {"tool": "get_pollex_storage_locations"}
    )
    if x == "error":
        raise
    return x

@tool
def get_materials_in_storage_location(storage_location: str) -> str: # fixed function
    """
    Requests materials from the system.
    Args:
        storage_location (str): The storage location to get materials from.
    Returns:
        str: A list of materials in the storage location.
    """
    
    x = global_input_function(
        {"tool": "get_materials_in_storage_location", "storage_location": storage_location}
    )
    if x == "error":
        raise
    return x

@tool
def get_storage_location(storage_location_id: str, plant: str) -> str: # fixed function
    """
    Requests information about a storage location.
    Args:
        storage_location_id (str): The id of the storage location to get information about.
        plant (str): The plant to get the storage location from.
    Returns:
        str: The information about the storage location.
    """
    
    x = global_input_function(
        {"tool": "get_storage_location", "storage_location_id": storage_location_id, "plant": plant}
    )
    if x == "error":
        raise
    return x

@tool
def create_storage_location(storage_location_id: str, description: str, plant: str) -> str: # fixed function
    """
    Creates a new storage location with the given description.
    Args:
        storage_location_id (str): The id of the storage location.
        description (str): The description of the storage location.
        plant (str): The plant to create the storage location in.
    Returns:
        str: A message indicating that the storage location was created.
    """
    
    x = global_input_function()
    if x != "":
        return x
    return 'Storage location with ID {} with description "{}" was created'.format(storage_location_id, description)

@tool
def send_email(email: str, content: str) -> str: # fixed function
    """
    Sends an email to the given email address with the given content.
    Args:
        email (str): The email address to send the email to.
        content (str): The content of the email.
    Returns:
        str: A message indicating that the email was sent.
    """
    return 'Email with content "{}" was send to {}'.format(content, email)

@tool
def transaction_SA38(program: str, program_parameters_dict: dict) -> str: # fixed function
    """
    Executes a transaction SA38 with the passed program and program parameters dictionary.
    Args:
        program (str): The program to execute.
        program_parameters_dict (dict): The parameters dictionary to pass to the program
    Returns:
        str: A message indicating of the transaction was successful.
    """
    
    x = global_input_function()
    if x != "":
        return x
    return "Transaction {} was successful with parameters: {}".format(program, program_parameters_dict)

@tool
def transaction_MMSC(material_id: str, storage_location_id: str, plant: str) -> str: # fixed function
    """
    Executes a transaction MMSC with the given material id, storage location id and plant.
    Args:
        material_id (str): The id of the material to move.
        storage_location_id (str): The id of the storage location to move the material to.
        plant (str): The plant to move the material in.
    Returns:
        str: A message indicating of the transaction was successful.
    """
    
    x = global_input_function()
    if x != "":
        return x
    return "Material {} was moved to storage location {} in plant {}".format(material_id, storage_location_id, plant)

@tool
def get_idoc_field(idoc_id: str, segment: str, key: str) -> str: # fixed function
    """
    Get the value of an IDoc field.
    Args:
        idoc_id (str): The id of the IDoc to get the field of.
        segment (str): The segment of the IDoc to get the field of.
        key (str): The key of the IDoc field to get the value of.
    Returns:
        str: The value of the IDoc field.
    """
    
    x = global_input_function(
        {"tool": "get_idoc_field", "idoc_id": idoc_id, "segment": segment, "key": key}
    )
    if x == "error":
        raise
    return x

@tool
def set_idoc_field(idoc_id: str, segment: str, key: str, value: str) -> str: # fixed function
    """
    Sets the value of an IDoc field.
    Args:
        idoc_id (str): The id of the IDoc to set the field of.
        segment (str): The segment of the IDoc to set the field of.
        key (str): The key of the IDoc field to get the value of.
        value (str): The value to set the IDoc field to.
    Returns:
        str: A message indicating if the IDoc field was set or not.
    """
    
    x = global_input_function()
    if x != "":
        return x
    return "IDoc field {} of segment {} of IDoc {} was set to {}\n".format(key, segment, idoc_id, value)

@tool
def reinitialize_idoc(idoc_id: str) -> str: # fixed function
    """
    Reinitializes an IDoc.
    Args:
        idoc_id (str): The id of the IDoc to reinitialize.
    Returns:
        str: A message indicating if the IDoc was reinitialized or not.
    """
    
    x = global_input_function()
    if x != "":
        return x
    return "IDoc {} was successfully reinitialized".format(idoc_id)

@tool
def get_current_date() -> str: # fixed function
    """
    Requests the current date.
    Returns:
        str: The current date.
    """
    
    x = global_input_function(
        {"tool": "get_current_date"}
    )
    if x == "error":
        raise
    return x

@tool
def rfc(function: str, rfc_parameters: dict) -> str: # function for complex API calls
    """
    Executes an RFC function with the given function and parameters.
    Args:
        function (str): The function to execute.
        rfc_parameters (dict): The parameters to pass to the function.
    Returns:
        str: A message indicating of the RFC function was successful.
    """
    
    x = global_input_function(
        {"tool": "rfc", "function": function, "rfc_parameters": rfc_parameters}
    )
    if x == "error":
        raise
    return x

@tool
def get_field_by_rpa(transaction: str, rpa_parameters: dict, field: str) -> str: # implemented RPA API
    """
    Retrieves field from an SAP transaction using RPA.
    Args:
        transaction (str): The transaction to get the entry from.
        rpa_parameters (dict): The parameters to pass to the transaction.
        field (str): The field to retrieve from the transaction.
    Returns:
        str: The entry from the SAP field.
    """
    
    x = global_input_function()
    if x == "error":
        raise
    return x

@tool
def retrieve_most_similar_rule(error_message_description: str) -> str: # most important function
    """
    Retrieves the most similar rule from the rule base for the given error message description.
    Args:
        error_message_description (str): Error message description to retrieve the most similar rule for.
    Returns:
        str: The most similar rule from the rule base.
    """
    global RETRIEVER
    result = RETRIEVER.invoke(error_message_description)[0]
    result = "Error Message Template:\n\n" + result.page_content + "\n\nRule:\n\n" + result.metadata.get("rule")
    return result

@tool
def generate_rule(user_request: str) -> str: # most important function
    """
    Generates a rule based on the user request.
    Args:
        user_request (str): The user request to generate the rule from.
    Returns:
        str: The generated rule.
    """
    llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            seed=42
    )
    response = llm.invoke(f"""You are an assistant that converts user requests into process steps an LLM agent will automatically execute to complete the process. 

    Convert the user_request into unambiguous and pragmatic step that include important step information, decision rules.

    - The steps need to be designed as clear, unambiguous instructions for an LLM agent.
    - Use if-else statements to mark decision points.
    - Nest every step of an if path within the if statement and every else path step within an else statement.
    - Hold the formatting of the steps as simple as possible.

    Convert the following user request into steps according to the instructions above:

    {user_request}""")
    process_rule = response.content
    token_usage = response.response_metadata.get("token_usage", {})
    input_tokens = token_usage.get("prompt_tokens", 0)
    cached_tokens = token_usage.get("prompt_tokens_details", {}).get("cached_tokens", 0)
    output_tokens = token_usage.get("completion_tokens", 0)
    
    global input_tokens_sum, cached_tokens_sum, output_tokens_sum, price
    input_tokens_sum += input_tokens
    cached_tokens_sum += cached_tokens
    output_tokens_sum += output_tokens
    price += (input_tokens - cached_tokens) * 250 / 1000000 + cached_tokens * 125 / 1000000 + output_tokens * 1000 / 1000000
    
    return process_rule

tools = [
    get_storage_locations_in_plant, 
    get_pollex_storage_locations, 
    get_materials_in_storage_location, 
    get_storage_location, 
    create_storage_location, 
    send_email, 
    transaction_SA38, 
    transaction_MMSC, 
    get_idoc_field, 
    get_current_date, 
    set_idoc_field, 
    reinitialize_idoc, 
    rfc, 
    get_field_by_rpa, 
    retrieve_most_similar_rule, 
    generate_rule
]

def create_react_agent(config, sys_msg, tools):

    llm = ChatOpenAI(
        **config
    )

    llm_with_tools = llm.bind_tools(tools)

    def assistant(state: MessagesState):
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

    builder = StateGraph(MessagesState)
    builder.add_node("assistant", assistant)
    builder.add_node("tools", ToolNode(tools))


    builder.add_edge(START, "assistant")
    builder.add_edge("tools", "assistant")
    builder.add_conditional_edges("assistant", tools_condition)

    react_graph = builder.compile()

    return react_graph

In [25]:
SYSTEM_PROMPT = """**Role:**  
You are SAPY an SAP agent responsible for handling user requests and resolving IDOC error messages.
You are only working in the plant 1540. 
Every user request or message you receive is related to this plant 1540.
You will receive the inputs containing either containing:
- error messages in the format below:
    Error Message:
    Idoc ID: <IDOC ID>
    Pollex ID: <Pollex ID>
    Description: <description>

- email responses in the format below:
    <email response content> # this is usually a response to a request you made, stating that the request was completed

    From: >Sender Name> <sender email>
    Date: <date>
    To: LLM Agent <llm.agent@company.com>
    Subject: <subject>

    <original email content>

- user requests with processes to complete:
    <user request content> # this is a request you need to complete (e.g. 'There was an issue with the IDoc 1234. Retrieve...' or 'Please compare the pollex storage locations with the storage locations in plant 1540.')

---

**Task:**
1. Analyze the input.
  - If it is an user request, call the `generate_rule`-tool and follow the generated rule to resolve the request.
  - If it is an email answer to your request stating that a request of you was completed, re-initialize the IDoc.
  - If it is an error message, retrieve the most similar rule to the error message description.
2. Compare the error message template and the rule. Reason step-by-step if this rule can really help you to resolve the original error provided in the input.
  - If the rule seems helpful, follow all steps of this rule to resolve the error efficiently using the tools provided.
    - Reason step-by-step how you can follow the rules and what tools you need to use to follow the rules. 
    - Include the the information from the error message or user request in your reasoning.
    - If a tool error occurs, reason step-by-step how you can solve the tool error and include the following steps:
      - Is the information already provided in the received input or error message?
      - Did you forget any parameters that are necessary for the tool? 
      - Are the parameters formatted correctly?
      - Can you use another tool to complete the step?
      - If the tool error occurs more than 3 times, send an email to error.handler1@company.com with the error message including the Idoc ID, Pollex ID, and Description and a request for help.
  - If you are not sure how to solve the error using the most similar rule or there is no similar rule found, send an email to including the Idoc ID, Pollex ID, and Description and a request for help to error.handler1@company.com

---"""

In [ ]:
results_dir = "results"
vector_db_dir = "vector_db"

data = {}
with open("user_requests.json", "r") as f:
    data = json.load(f)

config = {
    "model": "gpt-4o",
    "retriever_model": "text-embedding-3-small",
    "temperature": 0,
    "seed": 25,
}

embedder = OpenAIEmbeddings(
    model=config["retriever_model"],
)
db = FAISS.load_local(
    os.path.join(vector_db_dir, str(config["seed"])),
    embedder, 
    allow_dangerous_deserialization=True
)
RETRIEVER = db.as_retriever()

del config["retriever_model"]

sys_msg = SystemMessage(content=SYSTEM_PROMPT)
react_agent = create_react_agent(config=config, sys_msg=sys_msg, tools=tools)

results_file = f"simulation_results_rules_user_requests.csv"
results_path = os.path.join(results_dir, results_file)

simulated = []
try:
    simulation_results_df = pd.read_csv(results_path)
    simulated = simulation_results_df[["id", "path"]].copy()
    simulated = simulated.copy().values.tolist()
except Exception as e:
    simulation_results_df = pd.DataFrame(columns=["id", "query", "tasks", "gateways", "path", "results", "completed"])

def global_input_function(inp={}):
    global path_vars
    global idx
    
    recommendation = ""
    try:
        if "tool" in inp:
            additional_var = ""
            if int(idx) in [11, 12, 13, 14, 20, 21]:
                global is_locked
                while is_locked:
                    time.sleep(1)
                is_locked = True
                print(inp["tool"])
                x = input("Use alternative return value?")
                if x.lower() == "y":
                    additional_var = "_"
                else:
                    additional_var = ""
                is_locked = False
            if inp["tool"] == "get_storage_locations_in_plant":
                recommendation = path_vars["get_storage_locations_in_plant"][inp["plant"] + additional_var]
            elif inp["tool"] == "get_pollex_storage_locations":
                recommendation = path_vars["get_pollex_storage_locations"]
            elif inp["tool"] == "get_materials_in_storage_location":
                recommendation = path_vars["get_materials_in_storage_location"][inp["storage_location"] + additional_var]
            elif inp["tool"] == "get_storage_location":
                recommendation = path_vars["get_storage_location"][f"{inp['plant']},{inp['storage_location_id']}"]
            elif inp["tool"] == "get_idoc_field":
                recommendation = path_vars["get_idoc_field"][f"{inp['idoc_id']},{inp['segment']},{inp['key']}"]
            elif inp["tool"] == "get_current_date":
                recommendation = path_vars["get_current_date"]
            elif inp["tool"] == "rfc":
                table_name = inp['rfc_parameters']['TABLE_NAME']
                parameter_name = [x for x in inp['rfc_parameters'].keys() if x != "TABLE_NAME"][0]
                parameter = inp['rfc_parameters'][parameter_name]
                recommendation = path_vars["rfc"][f"{inp['function']},{table_name}"][f"{parameter_name},{parameter}"]
            else:
                recommendation = ""
    except Exception as e:
        recommendation = f"ERROR: {e}"
    x = recommendation
    return x

for i, row in enumerate(data):
    idx = row["id"]
    query = row["query"]
    tasks = row["tasks"]
    gateways = row["gateways"]
    tool_values = row["tool_values"]
    for path in tool_values:
        path_vars = {}
        path_vars = tool_values[path]
        if [int(row["id"]), path] in simulated:
            print(f"Skipping simulation for {row['id']} with path {path} as it was already simulated.")
            continue
        print(f"Processing request {idx}, {path}")
        is_locked = False
        results = ""
        input_tokens_sum = 0
        cached_tokens_sum = 0
        output_tokens_sum = 0
        price = 0
        clear_output()
        messages = [HumanMessage(content=query)]
        for event in react_agent.stream({"messages": messages}, stream_mode="values"):
            clear_output()
            for message in event["messages"]:
                print(message.pretty_repr())
            if type(event["messages"][-1]) == AIMessage:
                token_usage = event["messages"][-1].response_metadata.get("token_usage", {})
                input_tokens = token_usage.get("prompt_tokens", 0)
                cached_tokens = token_usage.get("prompt_tokens_details", {}).get("cached_tokens", 0)
                output_tokens = token_usage.get("completion_tokens", 0)
                input_tokens_sum += input_tokens
                cached_tokens_sum += cached_tokens
                output_tokens_sum += output_tokens
                price += (input_tokens - cached_tokens) * 250 / 1000000 + cached_tokens * 125 / 1000000 + output_tokens * 1000 / 1000000
        for message in event["messages"]:
            results += message.pretty_repr() + "\n"
        
        time.sleep(1)

        completed = ""
        x = input("Was the process completed and correct? (y/blank): ")
        if x.lower() == "y":
            completed = 1
        else:
            completed = 0
        results_row = [idx, query, tasks, gateways, path, input_tokens_sum, cached_tokens_sum, output_tokens_sum, price, results, completed]
        results_row = pd.DataFrame([results_row], columns=["id", "query", "tasks", "gateways", "path", "input_tokens", "cached_tokens", "output_tokens", "price", "results", "completed"])
        simulation_results_df = pd.concat([simulation_results_df, results_row], ignore_index=True).reset_index(drop=True)
        
        simulation_results_df.to_csv(results_path, index=False)